# q3dfit notebook: rest-frame MIR, JWST-MIRI/MRS data of F2M1106, restframe [SIV] 10.51 in Ch3B, from [Rupke et al. 2023](https://ui.adsabs.harvard.edu/abs/2023ApJ...953L..26R/abstract)

This Jupyter notebook allows you to run Q3Dfit, a PSF decomposition and spectral analysis package tailored for JWST NIRSpec and MIRI IFU observations. 

Q3Dfit is developed as a science-enabling data product by the Early Release Science Team #1335 Q3D. You can find more information about this ERS program **Q3D** [here](https://q3d.github.io/) and [here](https://www.stsci.edu/jwst/science-execution/approved-programs/dd-ers/program-1335).

The software is based on the existing package IFSFIT developed by Dave Rupke (see [ADS link](https://ui.adsabs.harvard.edu/abs/2017ApJ...850...40R/abstract)).

The following notebook will guide you through the initialization procedure and will then perform the analysis. 

## Table of Contents

* [1. Initialization](#chapter1)
    * [1.0. Setting up the directory tree](#chapter1_0)
    * [1.1. Initializing the fit](#chapter1_1)
    * [1.2. Setting up the data and models](#chapter1_2)
    * [1.3. Setting up the fitting parameters](#chapter1_3)
        * [1.3.1. Emission line parameters](#chapter1_3_1)
        * [1.3.2. Continuum parameters](#chapter1_3_2)
* [2. Run fit](#chapter2)
* [3. Plot fit results](#chapter3)
* [4. Combine fit results for all spaxels](#chapter4)
* [5. Plot science products](#chapter5)
* [6. Output science products to FITS files](#chapter6)

## 1. Initialization <a class="anchor" id="chapter1"></a>

In [ ]:
import os.path
import numpy as np
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
# Be sure to set the path to q3dfit correctly.
# For instance:
#import sys
#sys.path.append('/Users/jwstuser/q3dfit/')
#import sys
#sys.path.append("../")

### 1.0. Setting up the directory tree <a class="anchor" id="chapter1_0"></a>

Define the directories in which the data cube(s) that you want to analyse are stored and the output directories. We recommend creating a working directory that you name after your target, in which all outputs from q3dfit will be saved. Then download test data.

In [ ]:
# Base directory (book-keeping)
volume = 'f2m1106-miri-siv/'
# prefix label for output files
label = 'f2m1106-miri-siv'
# Input directory
indir = volume
if not os.path.exists(indir):
    os.makedirs(indir)
# Output directory
outdir = volume
if not os.path.exists(outdir):
    os.makedirs(outdir)
# Initialization file (q3di.npy) directory
initdir = volume
# Output logfile
logfile = outdir+label+'-fitlog.txt'

Download data from public Box folder:

In [ ]:
# make tuples of urls and download filenames
# infile = data cube
infile_tup=('https://rhodes.box.com/shared/static/oesd3kx96ec1grvf1sv4lxsr9n2wxztl.fits','f2m1106-miri-ch3b.fits')
# download files; by default don't force overwrite and take first element of output
from q3dfit.jnb import download_files
infile = download_files(infile_tup, outdir, force=False)[0]
# add subdirectory to filenames
infile = initdir+infile

### 1.1. Initializing the fit <a class="anchor" id="chapter1_1"></a>

The initial parameters of the fit are stored in an object of class `q3din`. Each parameter or attribute of this class controls some aspect of the fit process. We start by instantiating the class. The only required parameters at the outset are the input data cube and label; the label is used for output file naming. 

The default JWST pipeline output has data, variance, and data quality in extensions 1, 2, and 3, respectively. Our processed cube has a different set of extensions, so we specify them here.

In [ ]:
from q3dfit.q3din import q3din
q3di = q3din(infile, label, outdir=outdir, logfile=logfile)

Here's a list of the fit parameters that are automatically set:

In [ ]:
q3di.__dict__

### 1.2. Setting up the data and models <a class="anchor" id="chapter1_2"></a>

Some general information about your cube. `argsreadcube` is a dictionary of attributes sent to the `Cube` class.
- `q3dit` does calculations in f$_\lambda$ space, but assumes input units of MJy/sr, the JWST default. The output flux units will be in erg/s/cm$^2$/$\mu$m.
- The version of the JWST pipeline that was used for this reduction produced error spectra, rather than variance, which is assumed by `q3dfit.` The `error` flag fixes this.

In [ ]:
q3di.argsreadcube = {'fluxnorm': 1e-18,
                     'error': True}
cube = q3di.load_cube()

Let's plot a spaxel near the quasar to see how it looks. The arguments are column and row in unity-offset units. Specifying `radius=0` selects a single spaxel; specifying `radius` > 0 spaxels extracts flux in a circular aperture using `photutils`. The flux units are 10$^{-18}$ erg/s/cm$^2$/$\mu$m.

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=[10,4])
spec_test = cube.specextract(17, 28, radius=0)

Name and systemic redshift of the galaxy. `zsys_gas` is an input for calculating velocity maps in `q3dpro` and for initializing the arrays of initial guesses below.

In [ ]:
q3di.name = 'F2M1106'
q3di.zsys_gas = 0.435

Wavelength range over which to fit data. The user can also specify sets of regions to ignore in the fit.

In [ ]:
q3di.fitrange = [14.6,15.5]
#q3di.cutrange = np.array([,])

#### 1.3.1. Emission-line parameters <a class="anchor" id="chapter1_3_1"></a>

What lines do you want to fit? You can choose from the linelists available [here](https://github.com/Q3D/q3dfit/tree/main/q3dfit/data/linelists), or in `q3dfit/data/linelists/`.

In [ ]:
lines = ['[SIV]10.51']

This block sets up initial conditions for the emission-line fit to each spaxel. This initialization method adds a number of new attributes to the object. Emission lines are set to a common redshift and velocity dispersion, set to `q3di.zsys_gas` and 50 km/s by default. However, different sets of emission lines can have different velocities and linewidths by specifying different lines to which to tie particular emission lines. Different initial conditions can also be set on a spaxel-by-spaxel and/or line-by-line basis. The default number of velocity components is 1. Here, we change this to 3.

In [ ]:
q3di.init_linefit(lines, linetie='[SIV]10.51', maxncomp=2)
q3di.__dict__.keys()

Because of the complexity of these line profiles, we change the default initial conditions. `zinit_gas` and `siginit_gas` are dictionaries of arrays that hold the initial conditions for each line, spaxel, and velocity component. These were the initial conditions that were used to fit the data for Rupke et al. 2023.

In [ ]:
for i in lines:
    q3di.ncomp[i][22:,:] = 1
    q3di.ncomp[i][23:27,19:27] = 2
    q3di.ncomp[i][21:25,27:32] = 2
    q3di.zinit_gas[i][22:,:,0] = 0.435
    q3di.zinit_gas[i][23:27,19:27,0] = 0.437
    q3di.zinit_gas[i][23:27,19:27,1] = 0.433
    q3di.zinit_gas[i][21:25,27:32,0] = 0.437
    q3di.zinit_gas[i][21:25,27:32,1] = 0.433
    q3di.zinit_gas[i][:22,:,0] = 0.436
    q3di.zinit_gas[i][:22,:,1] = 0.439
    q3di.siginit_gas[i][:22,:,1] = 200.

`siglim_gas` sets lower and upper bounds for the Gaussian width (sigma) of the emission line. These limits can be set globablly, for all spaxels and components, by defining a 2-element array. The limits can also be set for individual spaxels (but all components) by defining an (Ncol x Nrow x 2) array.

In [ ]:
# Global limit
q3di.siglim_gas = np.array([5., 2000.])

# Spaxel-by-spaxel limit
# siglim_gas = np.ndarray((dx, dy, 2))
# siglim_gas[:,:,] = array([5.,1000.])
# siglim_gas[13, 10, :] = array([5.,500.])

The routine `checkcomp` automatically discards components that it deems insignificant after each fit. It does so with both a significance cut on flux, and if the linewidth is too large. If components are removed, the fit is re-run. The `sigcut` parameter determines the level of the significance cut. The `perror_useresid` option allows to substitute the formal line flux error with one estimated from the residual of the continnuum fit. This aids in more accurate component rejection in this case, because undersampling wiggles and fringeing are still present in the data and raise the actual error above that estimated by the pipeline (which is in any case too small). The `subone` option tells `checkcomp` to remove only one component at a time (necessary, e.g., if a two-component fit yields two low-significance components that both get rejected, but a one-component fit does not).

In [ ]:
q3di.checkcomp = True
q3di.argscheckcomp['sigcut'] = 3.
q3di.argscheckcomp['subone'] = True
q3di.perror_useresid = True

#### Line ratio constraints
Lines with ratios fixed by atomic physics have their ratios fixed automatically. Other line ratios can have bound constraints applied, or they can be fixed to a particular value.

`line1`, `line2`, and `comp` are required. `comp` is an array of velocity components (zero-indexed) on which to apply the constraints, one array for each pair of lines.

`value` is the initial value of `line1`/`line2`. Presently, if `value` is specified for one pair of lines, it must be specified for all. Otherwise, the initial value is determined from the data.

The ratio can be `fixed` to the initial value. Presently, if `fixed` is defined, it must be set to `True` or `False` for all pairs of line.

If the ratio is not `fixed`, `lower` and `upper` limits can also be specified. (If they are not, and the line pair is a doublet in the doublets.tbl file, then the lower and upper limits are set using the data in that file.) Presently, if `lower` or `upper` is defined here for one set of lines, it must be defined here for every pair of lines.

In [ ]:
# Required columns:
# line1 = ['[NI]5198', '[SII]6716']
# line2 = ['[NI]5200', '[SII]6731']
# comp = np.array([[0], [0]], dtype=np.int32)

# Optional columns:
# value = [1.5, 1.]
# fixed = [True, False]
# lower = []
# upper = []

# Write table
# from astropy.table import QTable
# lineratio = QTable([line1, line2, comp, value, fixed], names=['line1', 'line2', 'comp', 'value', 'fixed'])

# q3di.argslineinit['lineratio']=lineratio

#### Spectral resolution convolution

If no convolution is desired: do not set `spect_convol` (or set it to `{}`, or `None`).

If convolution is desired: `spect_convol` is a dictionary with two optional tags.
- `ws_instrum`: This specifies the desired convolution method. The syntax is: `{INSTRUMENT:[GRATING]}`. The values for `INSTRUMENT` and `GRATING` for pre-defined dispersion files should mirror the filename syntax in `q3dfit/data/dispersion_files/`. E.g., for file `jwst_miri_ch1a_disp.fits`, `INSTRUMENT=jwst_miri` and `GRATING=ch1a`. (Case is irrelevant. For convolution with a constant value of spectral resolution [R], Δλ FWHM in [$\mu$m], or velocity in [km/s], set `INSTRUMENT = flat` and `GRATING = ` a string containing `R`, `dlam`, or `dvel` and the corresponding numerical quantity. More thana one instrument and/or grating can be set.
- `dispdir`: Directory in which to find the dispersion files. If not set, the default `q3dfit` directory is searched.

Examples: 
1. flat R=500: `spect_instrum = {'flat':['R500']}`
2. flat velocity FWHM = 30km/s: `spect_instrum = {'flat':['dvel30']}`
3. flat Δλ FWHM = 4 Å: `spect_instrum = {'flat':['dlam0.0004']}`
4. JWST NIRSPEC / G140M: `spect_instrum = {'JWST_NIRSPEC':['G140M']}`
5. Spitzer IRS SH+LH: `spect_instrum = {'Spitzer_IRS':['ch1_sh','ch1_lh']}`

Note in the final example that two gratings are set.

In [ ]:
q3di.spect_convol['ws_instrum'] = {'JWST_MIRI':['ch3b']}

##### Creating convolution files (optional)

To create a dispersion file, use one of the following methods. The second two involve specific subclasses of the dispersion class used for the instrument/grating file or constant dispersion formats 

1. Create a `dispersion` object and use the `dispersion.write()` method. For example:

```
dispEx1 = dispersion()
dispEx1.write('/dispdir/disp.fits', wave=np.linspace(5.,10.,50), type='R', disp=np.full(50, 500.))
```

2. Create a `InstGratDispersion` object to attach instrument and grating information to the object and define the output filename in the `q3dfit` format. Use the `InstGratDispersion.writeInstGrat()` method.

```
dispEx2 = InstGratDispersion(`Keck_ESI`,`echellette`, dispdir=`/dispdir/`)
dispEx2.writeInstGrat(wave=np.linspace(5.,10.,50), type='dvel', disp=np.full(50, 30.))
```

3. Create a `FlatDispersion` object and use the `FlatDispersion.writeFlat()` method. This requires only a single value for the dispersion quantity and also defines the filename automatically.
```
dispEx3 = FlatDispersion(0.0004,`dlam`,wave=np.linspace(5.,10.,50))
dispEx3.writeFlat(dispdir=`/dispdir/`)
```

#### Options to `lmfit` and `scipy.optimize.least_squares`
`q3dfit` uses the `fit` method of the [`Model` class](https://lmfit.github.io/lmfit-py/model.html#lmfit.model.Model) of `lmfit` to call [`scipy.optimize.least_squares`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html). Both the method and function have options which can be changed in the `q3dfit` call. To do so, add key/value pairs to the `argslinefit` dictionary, which in turn is a keyword of the `q3di` dictionary.

The options to the `fit` method in `lmfit` that can currently be changed are the following:
- `max_nfev`: maximum number of function evaluations before the fit aborts
- `iter_cb`: if this is set to "per_iteration", the value of every model parameter at each function evaluation is printed to `stdout`

Most parameters of `least_squares` can be changed in this way, unless they are specifically set by `lmfit`. Examples which have been tested include:
- `x_scale`: jac
- `tr_solver`: lsmr
- `loss`: soft_l1
- `ftol`, `gtol`, `xtol`

In [ ]:
#q3di.argslinefit['iter_cb'] = 'per_iteration'
# As an example, to change the criteria for fit convergence from the defaults of 1.e-8 to 1.e-10:
q3di.argslinefit['method'] = 'leastsq'
q3di.argslinefit['ftol'] = 1.e-15
q3di.argslinefit['gtol'] = 1.e-15
q3di.argslinefit['xtol'] = 1.e-15
q3di.argslinefit['epsfcn'] = 1.e-10

#### 1.3.2 Continuum parameters <a class="anchor" id="chapter1_3_2"></a>

We next initialize the continuum. As part of this, we give it the name of our continuum fitting function, which in this case is a simple polynomial.

In [ ]:
q3di.init_contfit('fitpoly')
q3di.__dict__.keys()

`q3dfit` first masks emission lines before fitting. The default mask value is 500 km/s for each velocity component for the first fit. During the second fit, the mask value is set automatically using the best-fit linewidths determined from the first fit.

For this case, the lines are quite broad and we change the default.

In [ ]:
q3di.maskwidths_def = 1000.

The continuum fitting parameters specified here are for a polynomial.

In [ ]:
q3di.argscontfit['fitord'] = 3

If you want to run `q3dfit` in batch mode, run this cell, which saves q3di to an `npy` file. In your python command line, read in file and run `q3dfit` with
<pre><code>q3di = '/path/to/the/npy/file/q3di.npy'
from q3dfit.q3dfit import q3dfit
q3dfot(q3di,cols=cols,rows=rows)</code></pre>
N.B.: When running `q3dfit` using multiple cores (`ncores=N` in the call to `q3df`), the input dictionary has to be specified in this way; i.e., as a string describing the location of this .npy file.

In [ ]:
q3di_npy = 'q3di.npy'
np.save(initdir+q3di_npy, q3di)

## 2. Run fit <a class="anchor" id="chapter2"></a>

Choose columns and rows to fit. Ranges are specified as two-element lists specifying the first and last spaxel.

In [ ]:
# Example for individual spaxels
# spaxel with two components
cols = 18
rows = 28
# spaxel with no line
#cols=35
#rows=31
# Box encompassing extent of [SIV] emission
#cols = [12,35]
#rows = [16,35]

Run the fit. Choose `quiet=False` for verbose output. An output object for each spaxel, of class `q3dout`, is saved to a numpy binary file labeled with prefix `q3di['label']` and suffix `_col_row.npy`. See note above on multicore processing.

In [ ]:
from q3dfit.q3df import q3dfit
# Fit a single spaxel
q3dfit(q3di,cols=cols,rows=rows, quiet=False)
# Run all spaxels with multiple cores
#q3dfit(initdir+q3di_npy,cols=cols,rows=rows, ncores=10) #quiet=False)

## 3. Plot fit results <a class="anchor" id="chapter3"></a>

Load the output of a fit.

In [ ]:
q3di = np.load(initdir+'q3di.npy', allow_pickle=True).item()
cols = 18
rows = 28
from q3dfit.q3dout import load_q3dout
q3do = load_q3dout(q3di, cols, rows)

Set up the line plot parameters using a dictionary.

* `nx`: Number of subplots in the horizontal direction (default = 1)
* `ny`: Number of subplots in the vertical direction (default = 1)
* Required: choose one options for centerting the plot
    - `line`: a string list of line labels
    - `center_obs`: a float list of wavelengths of each subplot center, in the observed (plotted) frame
    - `center_rest`: a float list of wavelengths of each subplot center, in the rest frame, which are converted to obs. frame
* `size`: float list of widths in wavelength space of each subplot; if not specified (default = 300 $Å$)
* `IR`: set to `True` to use infrared-style plot

In [ ]:
argsplotline = dict()
argsplotline['nx'] = 1
argsplotline['ny'] = 1
argsplotline['line'] = ['[SIV]10.51']
argsplotline['size'] = [0.5]
argsplotline['figsize'] = [8,5]

Run the plot method. The output can be saved as a jpg by specifying `savefig=True`. A default filename is used, which can be overridden by specifying `outfile=file`. The output file will have the suffix `_lin` attached, so that the actual filename will be "file_lin.jpg".

In [ ]:
q3do.plot_line(q3di,plotargs=argsplotline)

The continuum plot can be changed by specifying several parameters. In this case, we have chosen to output a linear/linear plot of f$_\lambda$ vs. wavelength.

In [ ]:
argscontplot = dict()
argscontplot['xstyle'] = 'lin'
argscontplot['ystyle'] = 'lin'
argscontplot['fluxunit_out'] = 'flambda'
argscontplot['mode'] = 'dark'
argscontplot['figsize'] = [10,8]

Run two methods. The first computes the continuum values to plot, and the second does the plotting.

Because we specified `decompose_qso_fit=True` in the `q3di` object, three plots are created: one for the host-only light, one for quasar-only light, and one for the total continuum.

In [ ]:
q3do.sepcontpars(q3di)
q3do.plot_cont(q3di, plotargs=argscontplot)

## 4. Combine fit results from all spaxels.<a class="anchor" id="chapter4"></a>

This routine takes all of the spaxels you fit and combines the line- and continuum-fitting results together. The outputs are saved into two files. This example assumes that the spaxels listed here in each dimension have been fit.

In [ ]:
q3di = np.load(initdir+'q3di.npy', allow_pickle=True).item()
cols = [12,35]
rows = [16,35]
from q3dfit.q3dcollect import q3dcollect
q3dcollect(q3di, cols=cols, rows=rows, compsortpar='sigma', compsortdir='down')

## 5. Plot science products. <a class="anchor" id="chapter5"></a>

These routines take the output of `q3dcollect` and process them further for science output. The `q3dpro` class has methods to make maps of physical quantities.

In [ ]:
import q3dfit.q3dpro as q3dpro
qpro = q3dpro.Q3Dpro(q3di, NOCONT=True)

Start by plotting linemaps for [SIV], of both flux and velocity measures. The parameters listed below control the plotting.

In [ ]:
do_kpc = False
saveFile = False
flx = [1e-1,1e2]
qsocenter = [23.,27.]
pltarg = {'Ftot':flx,
          'Fci':flx,
          'Sig':[100.,850.],
          'v50':[-800.,800.],
          'w80':[100.,850.],
          'fluxlog': True}
qpro.make_linemap('[SIV]10.51', XYSTYLE=do_kpc, xyCenter=qsocenter,
                  SAVEDATA=saveFile, VMINMAX=pltarg, PLTNUM=1, CMAP='inferno')

Map of v50 over the cumulative velocity distribution:

In [ ]:
from q3dfit.q3dpro import OneLineData
s4data = OneLineData(qpro.linedat, '[SIV]10.51')
s4data.calc_cvdf(q3di.zsys_gas, [-1.5e3, 1.5e3], vstep=5)
s4data.make_cvdf_map(50., velran=[-1e3, 1e3], markcenter=[23.,27.],
                     outfile=True)

Map of W80 over the cumulative velocity distribution:

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=[6,4.5])
plt.imshow(s4data.calc_cvdf_vel(90, calc_from_posvel=False).T - s4data.calc_cvdf_vel(10, calc_from_posvel=False).T, 
       origin='lower', cmap='bwr', vmin=50, vmax=1500)
plt.colorbar()

## 6. Output science products to FITS files. <a class="anchor" id="chapter6"></a>

Export [SIV] map to a fits file:

In [ ]:
from q3dfit.q3dpro import save_to_fits
import copy
from astropy.io import fits
from photutils import centroids

cube_sum = np.sum(np.nan_to_num(cube.dat), axis=2).T

# get center
x1, y1 = centroids.centroid_2dg(cube_sum)
center_xy = [x1, y1]

# 2d header
newhead = copy.copy(cube.header_dat)
newhead['WCSAXES'] = 2
del newhead['CRPIX3']
del newhead['CRVAL3']
del newhead['CTYPE3']
del newhead['CUNIT3']
del newhead['CDELT3']
del newhead['PC1_3']
del newhead['PC2_3']
del newhead['PC3_1']
del newhead['PC3_2']
del newhead['PC3_3']
# new header
newhead['CRPIX1'] = x1+1.
newhead['CRPIX2'] = y1+1.
# Quasar centroid from SDSS
newhead['CRVAL1'] = 166.701361941
newhead['CRVAL2'] = 48.120086325
print(x1,y1)

newhead['BUNIT'] = 'erg/(s cm2 arcsec2)'

s4ftot = qpro.linedat.get_flux('[SIV]10.51')['flux']
s4ftot[s4ftot != np.nan] *= cube.fluxnorm/newhead['PIXAR_A2']

s4ftoterr = qpro.linedat.get_flux('[SIV]10.51')['fluxerr']
s4ftoterr[s4ftoterr != np.nan] *= cube.fluxnorm/newhead['PIXAR_A2']

# velocity map
s4v50 = s4data.calc_cvdf_vel(50, calc_from_posvel=False)

# sigma map
s4v84 = s4data.calc_cvdf_vel(84, calc_from_posvel=False)
s4v16 = s4data.calc_cvdf_vel(16, calc_from_posvel=False)
s4sig = s4v84 - s4v16

# Export [SIV] data into single file
s4file = outdir+'f2m1106_miri_s4.fits'
newhead['BUNIT'] = 'erg/(s cm2 arcsec2)'
newhead['EXTNAME'] = 'FLUX'
save_to_fits(s4ftot.T, newhead, s4file)
newhead['EXTNAME'] = 'ERR_FLUX'
fits.append(s4file, s4ftoterr.T, newhead)
newhead['BUNIT'] = 'km/s'
newhead['EXTNAME'] = 'V%50'
fits.append(s4file, s4v50.T, newhead)
newhead['EXTNAME'] = 'VSIG'
fits.append(s4file, s4sig.T, newhead)

Export line property maps by component

In [ ]:
line = '[SIV]10.51'
# No. of components
s4ncomp = qpro.linedat.get_ncomp(line)
s4file = outdir+'f2m1106_miri_s4_bycomp.fits'
newhead['EXTNAME'] = 'NCOMP'
save_to_fits(s4ncomp.T,newhead,s4file)
# Cycle through components
for i in np.arange(q3di.maxncomp):

    s4flx = qpro.linedat.get_flux(line,FLUXSEL=f'fc{i+1}')['flux']
    s4flx[s4flx != np.nan] *= cube.fluxnorm/newhead['PIXAR_A2']
    newhead['EXTNAME'] = f'FLX{i+1}'
    newhead['BUNIT'] = 'erg/(s cm2 arcsec2)'
    fits.append(s4file,s4flx.T,newhead)

    s4flxerr = qpro.linedat.get_flux(line,FLUXSEL=f'fc{i+1}')['fluxerr']
    s4flxerr[s4flxerr != np.nan] *= cube.fluxnorm/newhead['PIXAR_A2']
    newhead['EXTNAME'] = f'ERR_FLX{i+1}'
    fits.append(s4file,s4flxerr.T,newhead)

    s4wav = qpro.linedat.get_wave(line,COMPSEL=i+1)['wav']
    newhead['EXTNAME'] = f'WAV{i+1}'
    newhead['BUNIT'] = 'micron'
    fits.append(s4file,s4wav.T,newhead)

    s4waverr = qpro.linedat.get_wave(line,COMPSEL=i+1)['waverr']
    newhead['EXTNAME'] = f'ERR_WAV{i+1}'
    fits.append(s4file,s4waverr.T,newhead)

    s4sig = qpro.linedat.get_sigma(line,COMPSEL=i+1)['sig']
    newhead['EXTNAME'] = f'SIG{i+1}'
    newhead['BUNIT'] = 'km/s'
    fits.append(s4file,s4sig.T,newhead)
    
    s4sigerr = qpro.linedat.get_sigma(line,COMPSEL=i+1)['sigerr']
    newhead['EXTNAME'] = f'ERR_SIG{i+1}'
    fits.append(s4file,s4sigerr.T,newhead)